In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('energydata_complete.csv')

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [5]:
df.shape

(19735, 29)

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
new_df = df.drop(columns=['date','lights'])
normalised_df = pd.DataFrame(scaler.fit_transform(new_df), columns=new_df.columns)
target_variable = normalised_df['Appliances']

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(new_df, target_variable, test_size= 0.3 , random_state=42)

In [8]:
from sklearn.linear_model import LinearRegression 
linear_model = LinearRegression()
linear_model.fit(x_train,y_train)

LinearRegression()

In [9]:
predict_values = linear_model.predict(x_test)
predict_values

array([0.02803738, 0.07476636, 0.03738318, ..., 0.03738318, 0.09345794,
       0.03738318])

In [12]:
pd.DataFrame(y_test.head())

,Appliances
8980,0.028037
2754,0.074766
9132,0.037383
14359,0.037383
8875,0.056075


In [14]:
def get_weights_df(model, feat, col_name):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features',col_name]
    weights_df[col_name].round( 3 )
    return weights_df

In [15]:
linear_weight = get_weights_df(linear_model,x_train,'Linear Weight')

In [17]:
pd.DataFrame(linear_weight)

,Features,Linear Weight
0,T8,-2.598472e-18
1,T9,-1.683356e-18
2,T_out,-1.645511e-18
3,T3,-1.468314e-18
4,RH_7,-1.371580e-18
5,RH_3,-9.438869e-19
6,RH_1,-7.198866e-19
7,T4,-4.092373e-19
8,RH_out,-1.824185e-19
9,RH_5,-1.126185e-19


In [18]:
# Ridge Model 
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha= 0.4 )
ridge_reg.fit(x_train, y_train)

Ridge(alpha=0.4)

In [19]:
predict_values = ridge_reg.predict(x_test)

In [21]:
ridge_weight = get_weights_df(ridge_reg,x_train,'Ridge Weight')

In [22]:
ridge_weight

,Features,Ridge Weight
0,T9,-6.311427e-11
1,T2,-5.482075e-11
2,RH_2,-4.100955e-11
3,T_out,-3.304446e-11
4,RH_8,-1.724934e-11
5,RH_9,-5.261926e-12
6,RH_7,-5.052757e-12
7,T5,-4.782039e-12
8,RH_out,-3.262695e-12
9,T1,-1.109344e-12


In [23]:
# LASSO Model
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha= 0.001)
lasso_reg.fit(x_train,y_train)

Lasso(alpha=0.001)

In [25]:
lasso_weight = get_weights_df(lasso_reg,x_train,'Ridge Weight')

In [26]:
lasso_weight

,Features,Ridge Weight
0,T7,0.000000
1,Tdewpoint,0.000000
2,Visibility,0.000000
3,Windspeed,0.000000
4,RH_out,-0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,0.000000
9,RH_8,-0.000000


In [39]:
final_weights = pd.merge(linear_weight,ridge_weight,on='Features')
final_weights = pd.merge(final_weights,lasso_weight,on='Features')

In [40]:
final_weights

,Features,Linear Weight,Ridge Weight_x,Ridge Weight_y
0,T8,-2.598472e-18,2.982003e-11,0.000000
1,T9,-1.683356e-18,-6.311427e-11,0.000000
2,T_out,-1.645511e-18,-3.304446e-11,0.000000
3,T3,-1.468314e-18,7.711422e-11,0.000000
4,RH_7,-1.371580e-18,-5.052757e-12,-0.000000
5,RH_3,-9.438869e-19,1.433690e-11,0.000000
6,RH_1,-7.198866e-19,4.865175e-11,0.000000
7,T4,-4.092373e-19,8.335695e-12,0.000000
8,RH_out,-1.824185e-19,-3.262695e-12,-0.000000
9,RH_5,-1.126185e-19,7.687222e-13,0.000000


---

In [28]:
t2 = new_df.loc[:,'T2'].values
t6 = new_df.loc[:,'T6'].values

In [29]:
# Linear Model
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(t2, t6, test_size= 0.3 , random_state=42)

In [30]:
x_train = x_train.reshape(-1,1)
y_train = y_train.reshape(-1,1)
x_test = x_test.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [32]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(x_train,y_train)

LinearRegression()

In [33]:
predict_values = linear_model.predict(x_test)

In [35]:
#mean absoulute error
from sklearn import metrics
mae = metrics.mean_absolute_error(y_test,predict_values)
round(mae,2)

2.82

In [42]:
# r2 score
R2_score = metrics.r2_score(y_test,predict_values)
round(R2_score,2)

0.64

In [41]:
# RMSE
rmse = np.sqrt(metrics.mean_squared_error(y_test,predict_values))
round(rmse,3)

3.63